In [94]:
#//----------------------------
#//LIBRARIES
    #Math
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time
    #json library
import json
    #Analyst tools
import sys
sys.path.append('../')
from analysts_tools.growth import *
    #Procurement tools
from procurement_lib import send_slack_notification
from procurement_lib import redash
from analysts_tools.redash_methods import *

import random
import time

In [95]:
#//---------------------------------------------------------------------------------------------------------------------------
#// Parametros
"""Todays date should be the first day of the next month which is going to begin with SD"""
city_code = 'BOG'
    #year-month-day
todays_date = '2022-08-06' 
#non_buyers_date = '2022-07-06' #1 months
customer_date_desde = '2022-05-06' #1 months
    #data for sku sales data
#sku_date_desde = '2022-06-06' #2 months

t_cambio = 3000 #COL=3000 // MX=19.19 // BR=3.88

### Loading DATA_VENTAS // Removing SKUs /\ CUSTOMERS_ID

In [96]:
#Loading SELL'S DATA from a function
data_ventas = data_ventas_query(ciudad=city_code, tipo_cambio=t_cambio, desde=customer_date_desde, hasta = todays_date, tipo_negocio=1)

In [97]:
data_ventas = data_ventas.loc[data_ventas['name'].str.contains("j1", case=False)]

In [98]:
data_ventas_j1_darnel = data_ventas.loc[data_ventas['name'].str.contains("darnel", case=False)]
data_ventas_j1_wau = data_ventas.loc[data_ventas['name'].str.contains("wau", case=False)]

In [99]:
clientes_darnel = set(data_ventas_j1_darnel.customer_id.unique())
clientes_wau = set(data_ventas_j1_wau.customer_id.unique())
clientes_offer = clientes_darnel.difference(clientes_wau)

In [100]:
# import random 
import random

In [101]:
control = random.sample(clientes_offer, int(len(clientes_offer)*0.2))
test = clientes_offer.difference(control)

In [102]:
print(len(test), len(control), len(clientes_offer))

3188 797 3985


In [103]:
data_ventas[data_ventas['customer_id'].isin(control)].groupby(by=['name']).agg({'gmv_usd':np.sum, 'customer_id':pd.Series.nunique, 'cant':np.sum}).reset_index()

,name,gmv_usd,customer_id,cant
0,Contenedor J1 Darnel Paquete por 200 unidades...,18362.078778,390,895.98
1,Contenedor J1 Darnel Paquete x 20 unidades - ...,2071.409645,257,849.00
2,Contenedor J1 Darnel Paquete x 200 un - Paquete,23642.689567,420,1129.98
3,Contenedor J1 Kangupor x 200 und Paquete x 20...,1506.448664,55,72.50
4,Contenedor J1 Kangupor x 200 und Paquete x 20...,350.407000,4,17.00


In [104]:
data_ventas[data_ventas['customer_id'].isin(test)].groupby(by=['name']).agg({'gmv_usd':np.sum, 'customer_id':pd.Series.nunique, 'cant':np.sum}).reset_index()

,name,gmv_usd,customer_id,cant
0,Contenedor J1 Darnel Paquete por 200 unidades...,75312.663515,1608,3677.02
1,Contenedor J1 Darnel Paquete x 20 unidades - ...,9010.529572,1054,3690.19
2,Contenedor J1 Darnel Paquete x 200 un - Paquete,81687.244657,1660,3904.26
3,Contenedor J1 Kangupor x 200 und Paquete x 20...,7183.396156,231,345.50
4,Contenedor J1 Kangupor x 200 und Paquete x 20...,2144.475333,24,104.00


In [105]:
dict_clients = {'clientes': list(clientes_offer)}
df_clients = pd.DataFrame(dict_clients)
dict_control = {'control': list(control)}
df_control = pd.DataFrame(dict_control)
dict_test = {'test': list(test)}
df_test = pd.DataFrame(dict_test)

In [106]:
global_segment_list = [{'name':'20220806_BOG_all_Spend_differences_J1', 'customersIds': list([int(x) for x in test])}]

In [107]:
csv_file = {} 
csv_file['offer_name'] = '20220806_BOG_all_Spend_differences_J1'
csv_file['offer_description'] = None
csv_file['discount'] = 7
csv_file['automatically_consider_offer'] = "true"
csv_file['start_date'] = todays_date
csv_file['end_date'] = '2022-08-13'
csv_file['max_uses_per_order'] = 4
csv_file['max_uses_per_customer'] = 1
csv_file['customer_segment_id'] = None
csv_file['sku_id'] = 57791
global_offer_csv = pd.DataFrame(csv_file, index=[0])


In [108]:
# #//--------------------------------------------
# #//Exporting single file data
global_offer_csv.to_csv(f'SD/export_data/Below_Media/csv_{city_code}_All_MS_J1.csv', index = False)

df_clients.to_csv(f'SD/export_data/Below_Media/csv_{city_code}_clientes_offer_J1.csv', index = False)
df_control.to_csv(f'SD/export_data/Below_Media/csv_{city_code}_control_J1.csv', index = False)
df_test.to_csv(f'SD/export_data/Below_Media/csv_{city_code}_test_J1.csv', index = False)


with open(f'SD/export_data/Below_Media/nb_{city_code}_All_MS_J1.txt', 'w') as outfile:
        json.dump(global_segment_list, outfile)    
with open(f'SD/export_data/Below_Media/nb_{city_code}_All_MS_J1.json', 'w') as outfile:
        json.dump(global_segment_list, outfile)
print(f'SD/Done {city_code}')

SD/Done BOG
